In [2]:
import fiona
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
from geopandas import GeoDataFrame

datadir = "../../Data/Data_Frances/"

/Users/carlosmoreno/opt/anaconda3/envs/w210/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
def createranks2(df,columnby, nbins, namecolumn):

    df[columnby+"_Rank"] = df[columnby].rank(pct = True)
    bins = np.linspace(df[columnby+"_Rank"].min(), df[columnby+"_Rank"].max(),nbins+1)
    labels = list(range(1, nbins+1))
    df[namecolumn] = pd.cut(df[columnby+"_Rank"], bins=bins, labels=labels, include_lowest=True)
    df["Num"] = 1
    
    return df

In [4]:
def scale_group(row):
    if row >= 0.52: return(5)
    elif row >= 0.47: return(4)
    elif row >= 0.45: return(3)
    elif row >= 0.39: return(2)
    else: return(1)

In [5]:
dfall = pd.read_csv(datadir+"df_all.csv")

/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_39945/818637388.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  dfall = pd.read_csv(datadir+"df_all.csv")


In [6]:
dfall.head(1)

,Unnamed: 0.1,xg_key,xg_lon_t,xg_lat_t,xg_no_sh,xg_sh,xg_pred,rf_key,rf_lon_t,rf_lat_t,...,Y150,Y200,Y250,Y300,Y500,Y750,Y1000,Y1000plus,Ycoloc,geo_county2
0,0,033_174,-87.624327,30.880381,0.605934,0.394066,0,033_174,-87.624327,30.880381,...,0,0,0,0,0,0,0,101533,0,Escambia


In [7]:
list(dfall.columns)

['Unnamed: 0.1',
 'xg_key',
 'xg_lon_t',
 'xg_lat_t',
 'xg_no_sh',
 'xg_sh',
 'xg_pred',
 'rf_key',
 'rf_lon_t',
 'rf_lat_t',
 'rf_no_sh',
 'rf_sh',
 'rf_pred',
 'lr_key',
 'lr_lon_t',
 'lr_lat_t',
 'lr_no_sh',
 'lr_sh',
 'lr_pred',
 'geo_index',
 'geo_addy',
 'geo_addy_len',
 'geo_zipcode',
 'geo_city',
 'geo_county',
 'geo_key',
 'geo_lat',
 'geo_lon',
 'source',
 'Unnamed: 0',
 'Key',
 'Calcium Carbonate',
 'Gypsum',
 'Soil Health Organic Matter',
 'Percent Clay',
 'Percent Sand',
 'Percent Silt',
 'Available Water Storage',
 'date_ws',
 'name_x',
 'lon_t',
 'lat_t',
 'rolling_7_precip',
 'rolling_15_precip',
 'rolling_30_precip',
 'rolling_60_precip',
 'rolling_90_precip',
 'y1_mean_prc',
 'y1_max_prc',
 'y1_mean_tmp',
 'y1_max_tmp',
 'y1_min_tmp',
 'y2_mean_prc',
 'y2_max_prc',
 'y2_mean_tmp',
 'y2_max_tmp',
 'y2_min_tmp',
 'y3_mean_prc',
 'y3_max_prc',
 'y3_mean_tmp',
 'y3_max_tmp',
 'y3_min_tmp',
 'County',
 'county_fp',
 'name_y',
 'imagenum',
 'geometry',
 'AnnualCrop',
 'Fore

In [ ]:
# xg_no_sh, xg_sh, xg_pred

In [19]:
gbz = dfall.groupby(['geo_zipcode'])[['xg_no_sh', 'xg_sh']].mean()

In [20]:
gbz.head()

,xg_no_sh,xg_sh
geo_zipcode,,
05048,0.631184,0.368816
29229,0.536850,0.463150
31537,0.607343,0.392657
31562,0.627254,0.372746
31626,0.575480,0.424520


In [23]:
gbz.describe()

,xg_no_sh,xg_sh
count,978.000000,978.000000
mean,0.548203,0.451797
std,0.048020,0.048020
min,0.392747,0.366806
25%,0.520861,0.415524
50%,0.554785,0.445215
75%,0.584476,0.479139
max,0.633194,0.607253


In [28]:
gbz = gbz.reset_index()
gbz.head(2)

,index,geo_zipcode,xg_no_sh,xg_sh
0,0,05048,0.631184,0.368816
1,1,29229,0.536850,0.463150


In [35]:
gbz[gbz['geo_zipcode']== '05048']

,index,geo_zipcode,xg_no_sh,xg_sh
0,0,05048,0.631184,0.368816


In [38]:
zipcent[zipcent['ZIPCODE']== '05048']

,ZIPCODE,geometry,zip_lon,zip_lat


In [36]:
# for index, r in gbz.iterrows():
#     print(r.geo_zipcode)

In [37]:
# for index, r in zipcent.iterrows():
#     print(r.ZIPCODE)

In [26]:
gbz1 = pd.merge(gbz, zipcent, how="left", left_on='geo_zipcode', right_on='ZIPCODE')

In [39]:
gbz1.head(10)

,geo_zipcode,xg_no_sh,xg_sh,ZIPCODE,geometry,zip_lon,zip_lat
0,05048,0.631184,0.368816,NaN,NaN,NaN,NaN
1,29229,0.536850,0.463150,NaN,NaN,NaN,NaN
2,31537,0.607343,0.392657,NaN,NaN,NaN,NaN
3,31562,0.627254,0.372746,NaN,NaN,NaN,NaN
4,31626,0.575480,0.424520,NaN,NaN,NaN,NaN
5,31643,0.590081,0.409919,NaN,NaN,NaN,NaN
6,31792,0.609119,0.390881,NaN,NaN,NaN,NaN
7,32003,0.552314,0.447686,32003,POINT (-81.7123354920985 30.09798654800536),-81.712335,30.097987
8,32008,0.545975,0.454025,32008,POINT (-82.89853679731046 29.904964062711947),-82.898537,29.904964
9,32009,0.606728,0.393272,32009,POINT (-81.96817401559879 30.423994616367345),-81.968174,30.423995


In [41]:
gbz2 = gbz1[~gbz1["zip_lon"].isnull()]

In [42]:
len(gbz2)

907

In [43]:
gbz2.head(2)

,geo_zipcode,xg_no_sh,xg_sh,ZIPCODE,geometry,zip_lon,zip_lat
7,32003,0.552314,0.447686,32003,POINT (-81.7123354920985 30.09798654800536),-81.712335,30.097987
8,32008,0.545975,0.454025,32008,POINT (-82.89853679731046 29.904964062711947),-82.898537,29.904964


In [47]:
gbz20 = createranks2(gbz2,"xg_sh", 20, '20_bins')

/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_35299/3381852924.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columnby+"_Rank"] = df[columnby].rank(pct = True)
/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_35299/3381852924.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[namecolumn] = pd.cut(df[columnby+"_Rank"], bins=bins, labels=labels, include_lowest=True)
/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_35299/3381852924.py:7: SettingWithCopyWarning: 
A v

In [48]:
gbz20.head(2)

,geo_zipcode,xg_no_sh,xg_sh,ZIPCODE,geometry,zip_lon,zip_lat,xg_sh_Rank,20_bins,Num
7,32003,0.552314,0.447686,32003,POINT (-81.7123354920985 30.09798654800536),-81.712335,30.097987,0.512679,11,1
8,32008,0.545975,0.454025,32008,POINT (-82.89853679731046 29.904964062711947),-82.898537,29.904964,0.555678,12,1


In [50]:
gbz20["scale_zip"] = gbz20.apply(lambda r: scale_group(r.xg_sh), axis=1)

/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_35299/3892145527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gbz20["scale_zip"] = gbz20.apply(lambda r: scale_group(r.xg_sh), axis=1)


In [51]:
gbz20.head(2)

,geo_zipcode,xg_no_sh,xg_sh,ZIPCODE,geometry,zip_lon,zip_lat,xg_sh_Rank,20_bins,Num,scale_zip
7,32003,0.552314,0.447686,32003,POINT (-81.7123354920985 30.09798654800536),-81.712335,30.097987,0.512679,11,1,2
8,32008,0.545975,0.454025,32008,POINT (-82.89853679731046 29.904964062711947),-82.898537,29.904964,0.555678,12,1,3


In [52]:
gbz20.to_csv("dataae/zipscale.csv", index=False)

In [5]:
cols = list(dfall.head())

In [6]:
cols

['Unnamed: 0.1',
 'xg_key',
 'xg_lon_t',
 'xg_lat_t',
 'xg_no_sh',
 'xg_sh',
 'xg_pred',
 'rf_key',
 'rf_lon_t',
 'rf_lat_t',
 'rf_no_sh',
 'rf_sh',
 'rf_pred',
 'lr_key',
 'lr_lon_t',
 'lr_lat_t',
 'lr_no_sh',
 'lr_sh',
 'lr_pred',
 'geo_index',
 'geo_addy',
 'geo_addy_len',
 'geo_zipcode',
 'geo_city',
 'geo_county',
 'geo_key',
 'geo_lat',
 'geo_lon',
 'source',
 'Unnamed: 0',
 'Key',
 'Calcium Carbonate',
 'Gypsum',
 'Soil Health Organic Matter',
 'Percent Clay',
 'Percent Sand',
 'Percent Silt',
 'Available Water Storage',
 'date_ws',
 'name_x',
 'lon_t',
 'lat_t',
 'rolling_7_precip',
 'rolling_15_precip',
 'rolling_30_precip',
 'rolling_60_precip',
 'rolling_90_precip',
 'y1_mean_prc',
 'y1_max_prc',
 'y1_mean_tmp',
 'y1_max_tmp',
 'y1_min_tmp',
 'y2_mean_prc',
 'y2_max_prc',
 'y2_mean_tmp',
 'y2_max_tmp',
 'y2_min_tmp',
 'y3_mean_prc',
 'y3_max_prc',
 'y3_mean_tmp',
 'y3_max_tmp',
 'y3_min_tmp',
 'County',
 'county_fp',
 'name_y',
 'imagenum',
 'geometry',
 'AnnualCrop',
 'Fore

In [7]:
len(dfall["geo_zipcode"].unique())

979

In [8]:
len(dfall["geo_lat"].unique())

2064

In [18]:
dirzip = "../../Data/kx-florida-zip-code-areas-SHP/"
zipfile = "florida-zip-code-areas.shp"

zipdf = gpd.read_file(dirzip+zipfile)
zipdf.head(2)

,POST_OFFIC,AREA_SQ_MI,ZIPCODE,geometry
0,Sarasota,3.75,34236,"MULTIPOLYGON (((542009.554 369238.134, 541847...."
1,Sarasota,4.43,34239,"POLYGON ((548351.766 366830.778, 548330.648 36..."


In [17]:
gpd1 = GeoDataFrame(zipdf)

In [11]:
gpd2 = gpd1.to_crs(epsg=4326)

In [12]:
gpd2.head()

,POST_OFFIC,AREA_SQ_MI,ZIPCODE,geometry
0,Sarasota,3.75,34236,"MULTIPOLYGON (((-82.56211 27.32116, -82.56375 ..."
1,Sarasota,4.43,34239,"POLYGON ((-82.49819 27.29880, -82.49841 27.298..."
2,Sarasota,3.91,34242,"MULTIPOLYGON (((-82.55307 27.29630, -82.55337 ..."
3,Bradenton,16.24,34203,"POLYGON ((-82.46301 27.41185, -82.46311 27.411..."
4,Sarasota,6.56,34234,"POLYGON ((-82.53644 27.35227, -82.53653 27.351..."


In [ ]:
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000

In [13]:
zipnum = []
zipcent = []
ziplon = []
ziplat = []
for index, r in gpd2.iterrows():
    zipnum.append(r.ZIPCODE)
    zipcent.append(r.geometry.centroid)
    ziplon.append(r.geometry.centroid.x)
    ziplat.append(r.geometry.centroid.y)
zipcent = pd.DataFrame({"ZIPCODE": zipnum,
                        "geometry": zipcent,
                        "zip_lon": ziplon,
                        "zip_lat": ziplat})

/Users/carlosmoreno/opt/anaconda3/envs/w210/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [14]:
zipcent.head()

,ZIPCODE,geometry,zip_lon,zip_lat
0,34236,POINT (-82.55395859383323 27.330043687551672),-82.553959,27.330044
1,34239,POINT (-82.51940173815132 27.31094851596209),-82.519402,27.310949
2,34242,POINT (-82.54435472971103 27.263926323842465),-82.544355,27.263926
3,34203,POINT (-82.50769269749142 27.443248984182528),-82.507693,27.443249
4,34234,POINT (-82.53225124893119 27.367840134203064),-82.532251,27.367840


In [32]:
zipcent["ZIPCODE"].dtypes

dtype('O')

In [16]:
zipcent.to_csv("dataae/zipcentroid3.csv", index=False)

### Example from Web for reference

In [1]:
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame

df = pd.DataFrame({'zip':[19152,19047],
               'Lat':[40.058841,40.202162],
               'Lon':[-75.042164,-74.924594]})

geometry = [Point(xy) for xy in zip(df.Lon, df.Lat)]
gdf = GeoDataFrame(df, geometry=geometry)

gdf

/Users/carlosmoreno/opt/anaconda3/envs/w210/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


,zip,Lat,Lon,geometry
0,19152,40.058841,-75.042164,POINT (-75.04216 40.05884)
1,19047,40.202162,-74.924594,POINT (-74.92459 40.20216)


In [ ]:
gdf.set_crs(epsg=4326, inplace=True)

In [ ]:
# project to merkator
gdf.to_crs(epsg=3395)